Connected to Python

In [1]:
import polars as pl
import pandas as pd
import sqlite3
import os
import glob
import re
from PIL import Image
import io
from theFuzz import process

ModuleNotFoundError: No module named 'theFuzz'

In [2]:
import polars as pl
import pandas as pd
import sqlite3
import os
import glob
import re
from PIL import Image
import io
from thefuzz import process,fuzz

In [3]:
# create db connection
db_name = 'countries.db'

db_conn = sqlite3.connect(db_name)
db_cursor = sqlite3.Cursor(db_conn)

# read data from Wikipedia

In [4]:
capitals_url = 'https://en.wikipedia.org/wiki/List_of_national_capitals'
list_of_tables = pd.read_html(capitals_url)
capitals_df = list_of_tables[1]

cities_url = 'https://en.wikipedia.org/wiki/List_of_cities_by_elevation'
list_of_tables = pd.read_html(cities_url)
cities_df = list_of_tables[1]

In [5]:
capitals_df

,City/Town,Country/Territory,Continent,Notes
0,Abidjan (former capital; still hosts some gove...,Ivory Coast (Côte d'Ivoire),Africa,Abidjan is the largest city in Ivory Coast and...
1,Yamoussoukro (de jure),Ivory Coast (Côte d'Ivoire),Africa,Abidjan is the largest city in Ivory Coast and...
2,Abu Dhabi,United Arab Emirates,Asia,NaN
3,Abuja,Nigeria,Africa,Lagos was the capital from 1914 to 1991.
4,Accra,Ghana,Africa,NaN
...,...,...,...,...
257,Windhoek,Namibia,Africa,NaN
258,Yaoundé,Cameroon,Africa,NaN
259,Yaren (de facto),Nauru,Oceania,"Nauru has no official capital; however, the go..."
260,Yerevan,Armenia,Asia,NaN


In [6]:
# change column names
capitals_df = pl.from_pandas(capitals_df,rechunk=True)
capitals_df = capitals_df.rename({'City/Town':'city','Country/Territory':'country','Continent':'continent','Notes':'notes'})

In [7]:
cities_df = pl.from_pandas(cities_df,rechunk=True)
cities_df = cities_df.rename({'City Name/s':'city','Country/Territory':'country','Continental Region':'continent','Population':'population','Latitude':'latitude','Longitude':'longitude','Elevation (m)':'elevation'})
cities_df

country,city,continent,latitude,longitude,population,elevation
str,str,str,str,str,f64,str
"""Nepal""","""Pokhara""","""Asia""","""N28.2096""","""E83.9856""",523000.0,"""822"""
"""South Africa""","""Bloemfontein""","""Africa""","""S29.116667""","""E026.216667""",747431.0,"""1395"""
"""Andorra""","""Andorra la Vella""","""Europe""","""N42.5""","""W1.5""",22256.0,"""1023"""
"""China""","""Shanghai""","""Asia""","""N31.2304""","""E121.4737""",2.632e7,"""122"""
"""Italy""","""Milan""","""Europe""","""N45.4625""","""E9.186389""",1.378689e6,"""122"""
…,…,…,…,…,…,…
"""Australia""","""Brisbane""","""Oceania""","""S27.4710""","""E153.0242""",null,"""28"""
"""Australia""","""Adelaide""","""Oceania""","""S34.9286""","""E138.5986""",null,"""61"""
"""New Zealand""","""Wellington""","""Oceania""","""S41.2865""","""E174.7762""",null,"""20"""


In [8]:
# extract all the data before the extra information (brackets in this case)
# remove special characters from the city and country names so we can join two datasets later
capitals_df = capitals_df.with_columns(
    pl.col('city').str.replace(r'\s*\(.*', '').replace(r'[\',.]',''),
    pl.col('country').str.replace(r'\s*\(.*', '').replace(r'[\',.]',''),
    pl.col('city').str.extract(r'\s*\(.*',0).str.strip_chars(' ()').alias('city_extra_info')
)

cities_df = cities_df.with_columns(
    pl.col('city').str.replace(r'\s*\(.*', '').replace(r'[\',.]',''),
    pl.col('country').str.replace(r'\s*\(.*', '').replace(r'[\',.]',''),
    pl.col('elevation').str.replace(r'\s*\(.*', '').replace(r'[\',.]',''),

    )

countries_df = cities_df['country']

In [9]:
# join city and country data
capitals_with_coordinates = capitals_df.join(cities_df[['city','country','latitude','longitude','population','elevation']],how='left',on = ['city','country'])

# add combined data to database, schema on write
capitals_with_coordinates = capitals_with_coordinates.with_columns(pl.lit(None).alias('flag'))
capitals_with_coordinates.write_database(table_name='country_info',connection=f'sqlite:///{db_name}',if_table_exists='replace')

262

In [10]:
#create flags table
drop_table_sql = f'''DROP TABLE IF EXISTS flags;'''
create_table_sql = f'''CREATE TABLE flags (country VARCHAR(200), flag BLOB);'''
try:
   
    db_cursor.execute(drop_table_sql)
    db_cursor.execute(create_table_sql)
    db_conn.commit()
    print('Table created')
except:
    print('Cannot create table, check that you don"t have any errors in your code')

# add flags to database
path = 'flags'
flags_list = []
filenames = glob.glob(f'{path}/*.svg.png')
flags_dict = {}
# change the path slashes, unix to windows
filenames = [f.replace('\\','/') for f in filenames]

# load all the flags into the database
for file in filenames:
    try:
        with open(file,'rb') as f:
            # get rid of dots and square brackets, create country and flag pair
            flags_dict = {'country':re.split('[/.\\[\\]]',file)[1],'flag': f.read()}
        insert_data_sql = '''INSERT INTO flags (country,flag) VALUES (?,?);'''
        db_cursor.execute(insert_data_sql,(flags_dict['country'],flags_dict['flag']))
        print(f'added {flags_dict['country']}')
    except:
        print(f'Could not read {file}')


pd.read_sql('SELECT * FROM flags',db_conn)

Table created
added Abkhazia
added Afghanistan (Islamic Emirate of)
added Afghanistan (Islamic Republic of)
added Albania
added Algeria
added Andorra
added Angola
added Antigua and Barbuda
added Argentina
added Armenia
added Australia
added Austria
added Azerbaijan
added Bahamas
added Bahrain
added Bangladesh
added Barbados
added Belarus
added Belgium
added Belize
added Benin
added Bhutan
added Bolivia
added Bosnia and Herzegovina
added Botswana
added Brazil
added Brunei
added Bulgaria
added Burkina Faso
added Burundi
added Cambodia
added Cameroon
added Canada
added Cape Verde
added Central African Republic
added Chad
added Chile
added China
added Colombia
added Comoros
added Costa Rica
added Croatia
added Cuba
added Cyprus
added Czech Republic
added Democratic Republic of the Congo
added Denmark
added Djibouti
added Dominica
added Dominican Republic
added East Timor
added Ecuador
added Egypt
added El Salvador
added England
added Equatorial Guinea
added Eritrea
added Estonia
added Eswa

,country,flag
0,Abkhazia,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...
1,Afghanistan (Islamic Emirate of),b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...
2,Afghanistan (Islamic Republic of),b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...
3,Albania,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...
4,Algeria,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...
...,...,...
203,Vietnam,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...
204,Wales,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...
205,Yemen,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...
206,Zambia,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...


In [11]:
# create countries ref table, fuzzy match to fix issue with flags table
flags_df = pl.read_database('SELECT country FROM flags',db_conn)
countries_ref_df = capitals_df.select(['country'])

flags_df['country'] = countries_ref_df['country'].apply(lambda x: process.extractOne(x,countries_ref_df['country']),scorer=fuzz.partial_ratio)

AttributeError: 'Series' object has no attribute 'apply'

In [ ]:
try:
    join_tables = f'''UPDATE country_info SET flag = (SELECT flag FROM flags WHERE (country_info.country = flags.country))'''
    db_cursor.execute(join_tables)
    db_conn.commit()
    print('The country_info table ws updated with flags succesfully')
except:
    db_conn.rollback()
    print('Had to rollback, could not update country_info table with flags')

: 

In [ ]:
db_conn.close()

: 

In [12]:
x = 'England'

In [13]:
y = 'Engld'

In [14]:
fuzz.ratio(x,y)

83

In [15]:
fuzz.partial_ratio(x,y)

89

In [16]:
process.extract('England',x)

[('E', 90), ('n', 90), ('g', 90), ('l', 90), ('a', 90)]

In [17]:
process.extract('England',x,scorer=fuzz.ratio)

[('E', 25), ('n', 25), ('g', 25), ('l', 25), ('a', 25)]

In [18]:
process.extract('England',x,scorer=fuzz.partial_ratio)

[('E', 100), ('n', 100), ('g', 100), ('l', 100), ('a', 100)]

In [19]:
process.extract('England',y,scorer=fuzz.partial_ratio)

[('E', 100), ('n', 100), ('g', 100), ('l', 100), ('d', 100)]

In [20]:
x

'England'

In [21]:
process.extract('england',y,scorer=fuzz.partial_ratio)

[('E', 100), ('n', 100), ('g', 100), ('l', 100), ('d', 100)]

In [22]:
process.extract('england',x,scorer=fuzz.partial_ratio)

[('E', 100), ('n', 100), ('g', 100), ('l', 100), ('a', 100)]

In [23]:
len(process.extract('england',x,scorer=fuzz.partial_ratio))

5

In [24]:
len(process.extract('england',x,scorer=fuzz.ratio))

5

In [25]:
len(process.extract('engl',x,scorer=fuzz.ratio))

5

In [26]:
process.extract('engl',x,scorer=fuzz.ratio)

[('E', 40), ('n', 40), ('g', 40), ('l', 40), ('n', 40)]

In [27]:
x

'England'

In [28]:
x = ['England','engld','eng']

In [29]:
process.extract('engl',x,scorer=fuzz.ratio)

[('engld', 89), ('eng', 86), ('England', 73)]

In [30]:
process.extract('England',x,scorer=fuzz.ratio)

[('England', 100), ('engld', 83), ('eng', 60)]

In [31]:
x = ['England','engld','ENG']

In [32]:
process.extract('England',x,scorer=fuzz.ratio)

[('England', 100), ('engld', 83), ('ENG', 60)]

In [33]:
process.extract('England',x,scorer=fuzz.partial_ratio)

[('England', 100), ('ENG', 100), ('engld', 89)]

In [34]:
x = ['England','The offical state of England','Englaend']

In [35]:
x = ['England','The offical state of England','Englaend']

In [36]:
process.extract('England',x,scorer=fuzz.partial_ratio)

[('England', 100), ('The offical state of England', 100), ('Englaend', 86)]

In [37]:
process.extract('England',x,scorer=fuzz.ratio)

[('England', 100), ('Englaend', 93), ('The offical state of England', 40)]

In [38]:
process.extract('England',x,scorer=fuzz.token_ratio)

AttributeError: module 'thefuzz.fuzz' has no attribute 'token_ratio'

In [39]:
process.extract('England',x,scorer=fuzz.token)

AttributeError: module 'thefuzz.fuzz' has no attribute 'token'

In [40]:
process.extract('England',x,scorer=fuzz.token_set_ratio)

[('England', 100), ('The offical state of England', 100), ('Englaend', 93)]

In [41]:
x = ['England','The offical state of England','Englaend','Eng land']

In [42]:
process.extract('England',x,scorer=fuzz.token_set_ratio)

[('England', 100),
 ('The offical state of England', 100),
 ('Englaend', 93),
 ('Eng land', 93)]

In [43]:
process.extract('England',x,scorer=fuzz.partial_ratio)

[('England', 100),
 ('The offical state of England', 100),
 ('Englaend', 86),
 ('Eng land', 86)]

In [44]:
process.extractOne('England',x,scorer=fuzz.partial_ratio)

('England', 100)

In [45]:
countries_ref_df['country'].select(process.extractOne(countries_ref_df['country']),scorer=fuzz.token_set_ratio)

AttributeError: 'Series' object has no attribute 'select'

In [46]:
countries_ref_df.select(process.extractOne(countries_ref_df),scorer=fuzz.token_set_ratio)

TypeError: extractOne() missing 1 required positional argument: 'choices'

In [47]:
countries_ref_df.select(process.extractOne(countries_ref_df,scorer=fuzz.token_set_ratio))

TypeError: extractOne() missing 1 required positional argument: 'choices'

In [48]:
countries_ref_df.select(process.extractOne(flags_df,countries_ref_df,scorer=fuzz.token_set_ratio))

TypeError: sentence must be a String

In [49]:
countries_ref_df.select(process.extractOne(flags_df['country'],countries_ref_df,scorer=fuzz.token_set_ratio))

TypeError: sentence must be a String

In [50]:
flags_df['country']

country
str
"""Abkhazia"""
"""Afghanistan (Islamic Emirate o…"
"""Afghanistan (Islamic Republic …"
"""Albania"""
"""Algeria"""
…
"""Vietnam"""
"""Wales"""
"""Yemen"""


In [51]:
countries_ref_df

country
str
"""Ivory Coast"""
"""Ivory Coast"""
"""United Arab Emirates"""
"""Nigeria"""
"""Ghana"""
…
"""Namibia"""
"""Cameroon"""
"""Nauru"""


In [52]:
countries_ref_df

country
str
"""Ivory Coast"""
"""Ivory Coast"""
"""United Arab Emirates"""
"""Nigeria"""
"""Ghana"""
…
"""Namibia"""
"""Cameroon"""
"""Nauru"""


In [53]:
countries_ref_df.select(process.extractOne(countries_ref_df,flags_df['country'],countries_ref_df,scorer=fuzz.token_set_ratio)).alias('new_column')

TypeError: the truth value of a DataFrame is ambiguous

Hint: to check if a DataFrame contains any values, use `is_empty()`.

In [54]:
countries_ref_df.select(process.extractOne(countries_ref_df,flags_df['country'],countries_ref_df['country'],scorer=fuzz.token_set_ratio))

TypeError: the truth value of a Series is ambiguous

Here are some things you might want to try:
- instead of `if s`, use `if not s.is_empty()`
- instead of `s1 and s2`, use `s1 & s2`
- instead of `s1 or s2`, use `s1 | s2`
- instead of `s in [y, z]`, use `s.is_in([y, z])`


In [55]:
countries_ref_df.select(process.extractOne(countries_ref_df,flags_df['country'].str,countries_ref_df['country'].str,scorer=fuzz.token_set_ratio))

TypeError: 'StringNameSpace' object is not callable

In [56]:
countries_ref_df.select(process.extractOne(countries_ref_df['country'].str,flags_df['country'].str,scorer=fuzz.token_set_ratio))

TypeError: sentence must be a String

In [57]:
countries_ref_df['country'].str

In [58]:
countries_ref_df['country']

country
str
"""Ivory Coast"""
"""Ivory Coast"""
"""United Arab Emirates"""
"""Nigeria"""
"""Ghana"""
…
"""Namibia"""
"""Cameroon"""
"""Nauru"""


In [59]:
countries_ref_df.select(process.extractOne(countries_ref_df['country'].str,flags_df['country'].str,scorer=fuzz.token_set_ratio).alias('fuzzy_country'))

TypeError: sentence must be a String

In [60]:
countries_ref_df['country'].str.capitalize()

AttributeError: 'StringNameSpace' object has no attribute 'capitalize'

In [61]:
countries_ref_df['country'].str.capitalize

AttributeError: 'StringNameSpace' object has no attribute 'capitalize'

In [62]:
countries_ref_df['country'].cast(pl.Utf8).str.capitalize()

AttributeError: 'StringNameSpace' object has no attribute 'capitalize'

In [63]:
countries_ref_df['country'].cast(pl.Utf16).str.capitalize()

AttributeError: module 'polars' has no attribute 'Utf16'

In [64]:
countries_ref_df['country'].cast(pl.Utf8).str.capitalize()

AttributeError: 'StringNameSpace' object has no attribute 'capitalize'

In [65]:
countries_ref_df['country'].cast(pl.Utf8).str.count()

AttributeError: 'StringNameSpace' object has no attribute 'count'

In [66]:
countries_ref_df['country'].str()

TypeError: 'StringNameSpace' object is not callable

In [67]:
countries_ref_df['country'].str

In [68]:
countries_ref_df['country']

country
str
"""Ivory Coast"""
"""Ivory Coast"""
"""United Arab Emirates"""
"""Nigeria"""
"""Ghana"""
…
"""Namibia"""
"""Cameroon"""
"""Nauru"""


In [69]:
countries_ref_df['country'].select(
    [
        pl.col('country').str.capitalize().alias('country')
    ]
)

AttributeError: 'Series' object has no attribute 'select'

In [70]:
countries_ref_df.select(
    [
        pl.col('country').str.capitalize().alias('country')
    ]
)

AttributeError: 'ExprStringNameSpace' object has no attribute 'capitalize'

In [71]:
countries_ref_df.select(
    [
        pl.col('country').str.count().alias('country')
    ]
)

AttributeError: 'ExprStringNameSpace' object has no attribute 'count'

In [72]:
countries_ref_df.select(
    [
        pl.col('country').str.alias('country')
    ]
)

AttributeError: 'ExprStringNameSpace' object has no attribute 'alias'

In [73]:
dir(countries_ref_df['country'].str)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_accessor',
 '_s',
 'concat',
 'contains',
 'contains_any',
 'count_matches',
 'decode',
 'encode',
 'ends_with',
 'explode',
 'extract',
 'extract_all',
 'extract_groups',
 'extract_many',
 'find',
 'head',
 'join',
 'json_decode',
 'json_path_match',
 'len_bytes',
 'len_chars',
 'pad_end',
 'pad_start',
 'replace',
 'replace_all',
 'replace_many',
 'reverse',
 'slice',
 'split',
 'split_exact',
 'splitn',
 'starts_with',
 'strip_chars',
 'strip_chars_end',
 'strip_chars_start',
 'strip_prefix',
 'strip_suffix',
 'strptime',
 'tail',
 'to_date',
 'to_datetime',
 'to_decimal',
 'to_integer',
 'to_lowerca

In [74]:
countries_ref_df.select(
    [
        pl.col('country').str.to_uppercase().alias('country')
    ]
)

country
str
"""IVORY COAST"""
"""IVORY COAST"""
"""UNITED ARAB EMIRATES"""
"""NIGERIA"""
"""GHANA"""
…
"""NAMIBIA"""
"""CAMEROON"""
"""NAURU"""


In [75]:
countries_ref_df.with_columns(pl.col(countries_ref_df['country']).apply(lambda x: process.extractOne(x,flags_df['country'].str,scorer=fuzz.token_set_ratio).alias('fuzzy_country')))

AttributeError: 'Expr' object has no attribute 'apply'

In [76]:
countries_ref_df.with_columns(pl.col('country').apply(lambda x: process.extractOne(x,flags_df['country'].str,scorer=fuzz.token_set_ratio).alias('fuzzy_country')))

AttributeError: 'Expr' object has no attribute 'apply'

In [77]:
countries_ref_df.with_column(pl.col('country').apply(lambda x: process.extractOne(x,flags_df['country'].str,scorer=fuzz.token_set_ratio).alias('fuzzy_country')))

AttributeError: 'DataFrame' object has no attribute 'with_column'

In [78]:
countries_ref_df.with_columns(pl.col('country').apply(lambda x: process.extractOne(x,flags_df['country'],scorer=fuzz.token_set_ratio).alias('fuzzy_country')))

AttributeError: 'Expr' object has no attribute 'apply'

In [79]:
countries_ref_df.with_columns(pl.col('country').alias('fuzzy_country')

SyntaxError: incomplete input (<ipython-input-79-7eb6b838cffe>, line 1)

In [80]:
countries_ref_df.with_columns(pl.col('country').alias('fuzzy_country'))

country,fuzzy_country
str,str
"""Ivory Coast""","""Ivory Coast"""
"""Ivory Coast""","""Ivory Coast"""
"""United Arab Emirates""","""United Arab Emirates"""
"""Nigeria""","""Nigeria"""
"""Ghana""","""Ghana"""
…,…
"""Namibia""","""Namibia"""
"""Cameroon""","""Cameroon"""
"""Nauru""","""Nauru"""


In [81]:
countries_ref_df.with_columns(pl.col('country').apply(lambda x: x).alias('fuzzy_country'))

AttributeError: 'Expr' object has no attribute 'apply'

In [82]:
countries_ref_df.with_columns(pl.col('country').select(lambda x: x).alias('fuzzy_country'))

AttributeError: 'Expr' object has no attribute 'select'

In [83]:
countries_ref_df.with_columns(pl.col('country').apply(lambda x: x).alias('fuzzy_country'))

AttributeError: 'Expr' object has no attribute 'apply'

In [84]:
countries_ref_df.with_columns(pl.col('country').apply(1).alias('fuzzy_country'))

AttributeError: 'Expr' object has no attribute 'apply'

In [85]:
countries_ref_df

country
str
"""Ivory Coast"""
"""Ivory Coast"""
"""United Arab Emirates"""
"""Nigeria"""
"""Ghana"""
…
"""Namibia"""
"""Cameroon"""
"""Nauru"""


In [86]:
countries_ref_df.apply(pl.col('country').alias('fuzzy_country'))

AttributeError: 'DataFrame' object has no attribute 'apply'

In [87]:
countries_ref_df.apply(lambda x: (x + x).alias('fuzzy_country'))

AttributeError: 'DataFrame' object has no attribute 'apply'

In [88]:
type(countries_ref_df)

polars.dataframe.frame.DataFrame

In [89]:
countries_ref_df.select(pl.col('country'))

country
str
"""Ivory Coast"""
"""Ivory Coast"""
"""United Arab Emirates"""
"""Nigeria"""
"""Ghana"""
…
"""Namibia"""
"""Cameroon"""
"""Nauru"""


In [90]:
countries_ref_df['country'].map_rows(lambda x: (x + x).alias('fuzzy_country'))

AttributeError: 'Series' object has no attribute 'map_rows'

In [91]:
countries_ref_df.map_rows(lambda x: (x + x).alias('fuzzy_country'))

AttributeError: 'tuple' object has no attribute 'alias'

In [92]:
countries_ref_df.map_rows(lambda x: x + x.alias('fuzzy_country'))

AttributeError: 'tuple' object has no attribute 'alias'

In [93]:
countries_ref_df

country
str
"""Ivory Coast"""
"""Ivory Coast"""
"""United Arab Emirates"""
"""Nigeria"""
"""Ghana"""
…
"""Namibia"""
"""Cameroon"""
"""Nauru"""


In [94]:
countries_ref_df.map_rows(lambda x: x )

column_0
str
"""Ivory Coast"""
"""Ivory Coast"""
"""United Arab Emirates"""
"""Nigeria"""
"""Ghana"""
…
"""Namibia"""
"""Cameroon"""
"""Nauru"""


In [95]:
countries_ref_df.map_rows(lambda x: x.str.to_uppercase())

AttributeError: 'tuple' object has no attribute 'str'

In [96]:
countries_ref_df

country
str
"""Ivory Coast"""
"""Ivory Coast"""
"""United Arab Emirates"""
"""Nigeria"""
"""Ghana"""
…
"""Namibia"""
"""Cameroon"""
"""Nauru"""


In [97]:
countries_ref_df[0]

country
str
"""Ivory Coast"""


In [98]:
type(countries_ref_df[0])

polars.dataframe.frame.DataFrame

In [99]:
type(countries_ref_df[0,0])

str

In [100]:
countries_ref_df.map_rows(lambda x: x.to_uppercase())

AttributeError: 'tuple' object has no attribute 'to_uppercase'

In [101]:
countries_ref_df.map_rows(lambda x: x + x)

column_0,column_1
str,str
"""Ivory Coast""","""Ivory Coast"""
"""Ivory Coast""","""Ivory Coast"""
"""United Arab Emirates""","""United Arab Emirates"""
"""Nigeria""","""Nigeria"""
"""Ghana""","""Ghana"""
…,…
"""Namibia""","""Namibia"""
"""Cameroon""","""Cameroon"""
"""Nauru""","""Nauru"""


In [102]:
countries_ref_df.map_rows(lambda x: x.str + x.str)

AttributeError: 'tuple' object has no attribute 'str'

In [103]:
countries_ref_df.map_rows(lambda x: type(x))

RuntimeError: BindingsError: "Could not determine output type"

In [104]:
countries_ref_df.map_rows(lambda x: x)

column_0
str
"""Ivory Coast"""
"""Ivory Coast"""
"""United Arab Emirates"""
"""Nigeria"""
"""Ghana"""
…
"""Namibia"""
"""Cameroon"""
"""Nauru"""


In [105]:
countries_ref_df.map_rows(lambda x: upper(x))

NameError: name 'upper' is not defined

In [106]:
countries_ref_df.map_rows(lambda x: x+x)

column_0,column_1
str,str
"""Ivory Coast""","""Ivory Coast"""
"""Ivory Coast""","""Ivory Coast"""
"""United Arab Emirates""","""United Arab Emirates"""
"""Nigeria""","""Nigeria"""
"""Ghana""","""Ghana"""
…,…
"""Namibia""","""Namibia"""
"""Cameroon""","""Cameroon"""
"""Nauru""","""Nauru"""


In [107]:

countries_ref_df.map_rows(lambda x: x[1:])

shape: (0, 0)
┌┐
╞╡
└┘

In [108]:

countries_ref_df.map_rows(lambda x: x[2:])

shape: (0, 0)
┌┐
╞╡
└┘

In [109]:

countries_ref_df.map_rows(lambda x: x[2])

IndexError: tuple index out of range

In [110]:

countries_ref_df.map_rows(lambda x: (x[2]))

IndexError: tuple index out of range

In [111]:

countries_ref_df.map_rows(lambda x: (x[1]))

IndexError: tuple index out of range

In [112]:

countries_ref_df.map_rows(lambda x: (x[0]))

map
str
"""Ivory Coast"""
"""Ivory Coast"""
"""United Arab Emirates"""
"""Nigeria"""
"""Ghana"""
…
"""Namibia"""
"""Cameroon"""
"""Nauru"""


In [113]:
countries_ref_df.map_rows(lambda x: (x[0][1:]))

map
str
"""vory Coast"""
"""vory Coast"""
"""nited Arab Emirates"""
"""igeria"""
"""hana"""
…
"""amibia"""
"""ameroon"""
"""auru"""


In [114]:
countries_ref_df.map_rows(lambda x: (x[0]+x[0]))

map
str
"""Ivory CoastIvory Coast"""
"""Ivory CoastIvory Coast"""
"""United Arab EmiratesUnited Ara…"
"""NigeriaNigeria"""
"""GhanaGhana"""
…
"""NamibiaNamibia"""
"""CameroonCameroon"""
"""NauruNauru"""


In [115]:
countries_ref_df.map_rows(lambda x: (x[0]+x[0]).alias('new_country'))

AttributeError: 'str' object has no attribute 'alias'

In [116]:
countries_ref_df.map_rows(lambda x: (x[0]+x[0])).alias('new_country')

AttributeError: 'DataFrame' object has no attribute 'alias'

In [117]:
countries_ref_df.map_rows(lambda x: process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio))

column_0,column_1
str,i64
"""Ivory Coast""",100
"""Ivory Coast""",100
"""United Arab Emirates""",100
"""Nigeria""",100
"""Ghana""",100
…,…
"""Namibia""",100
"""Cameroon""",100
"""Nauru""",100


In [118]:
countries_ref_df.map_rows(lambda x: process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)).filter('column_1' < 100)

TypeError: '<' not supported between instances of 'str' and 'int'

In [119]:
countries_ref_df.map_rows(lambda x: process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)).filter(pl.col('column_1') < 100)

column_0,column_1
str,i64
"""Solomon Islands""",64
"""Bosnia and Herzegovina""",56
"""Solomon Islands""",74
"""Sierra Leone""",64
"""Sierra Leone""",64
…,…
"""Marshall Islands""",69
"""Marshall Islands""",70
"""Angola""",71


In [120]:
countries_ref_df.map_rows(lambda x: process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio))

column_0,column_1
str,i64
"""Ivory Coast""",100
"""Ivory Coast""",100
"""United Arab Emirates""",100
"""Nigeria""",100
"""Ghana""",100
…,…
"""Namibia""",100
"""Cameroon""",100
"""Nauru""",100


In [121]:
countries_ref_df.map_rows((lambda x: process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)).alias(['fuzzy_country','score']))

AttributeError: 'function' object has no attribute 'alias'

In [122]:
countries_ref_df.map_rows(lambda x: process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio).with_columns(pl.col('column_0').alias('fuzzy_country'))

SyntaxError: incomplete input (<ipython-input-122-8d8827807171>, line 1)

In [123]:
countries_ref_df.map_rows(lambda x: process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio).with_columns(pl.col('column_0').alias('fuzzy_country')))

AttributeError: 'tuple' object has no attribute 'with_columns'

In [124]:
countries_ref_df.map_rows(lambda x: process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio).select(pl.col('column_0'))

SyntaxError: incomplete input (<ipython-input-124-8669a0603ec9>, line 1)

In [125]:
countries_ref_df.map_rows(lambda x: process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio).select(pl.col('column_0')))

AttributeError: 'tuple' object has no attribute 'select'

In [126]:
countries_ref_df.map_rows(lambda x: process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)).select(pl.col('column_0'))

column_0
str
"""Ivory Coast"""
"""Ivory Coast"""
"""United Arab Emirates"""
"""Nigeria"""
"""Ghana"""
…
"""Namibia"""
"""Cameroon"""
"""Nauru"""


In [127]:
countries_ref_df.map_rows(lambda x: process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)).with_columns(pl.col('column_0').alias('fuzzy_country'))

column_0,column_1,fuzzy_country
str,i64,str
"""Ivory Coast""",100,"""Ivory Coast"""
"""Ivory Coast""",100,"""Ivory Coast"""
"""United Arab Emirates""",100,"""United Arab Emirates"""
"""Nigeria""",100,"""Nigeria"""
"""Ghana""",100,"""Ghana"""
…,…,…
"""Namibia""",100,"""Namibia"""
"""Cameroon""",100,"""Cameroon"""
"""Nauru""",100,"""Nauru"""


In [128]:
countries_ref_df.map_rows(lambda x: process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)).rename(pl.col('column_0'),'fuzzy_country')

TypeError: DataFrame.rename() takes 2 positional arguments but 3 were given

In [129]:
countries_ref_df.map_rows(lambda x: process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)).rename(pl.col'column_0','fuzzy_country')

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-129-880e81125015>, line 1)

In [130]:
countries_ref_df.map_rows(lambda x: process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)).rename(pl.col()'column_0'),'fuzzy_country')

SyntaxError: unmatched ')' (<ipython-input-130-26ac4ae5b885>, line 1)

In [131]:
countries_ref_df.map_rows(lambda x: process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)).rename(pl.col('column_0'),'fuzzy_country')

TypeError: DataFrame.rename() takes 2 positional arguments but 3 were given

In [132]:
countries_ref_df.map_rows(lambda x: process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)).select(pl.col('column_0').rename('fuzzy_country'))

AttributeError: 'Expr' object has no attribute 'rename'

In [133]:
countries_ref_df.map_rows(lambda x: process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio))

column_0,column_1
str,i64
"""Ivory Coast""",100
"""Ivory Coast""",100
"""United Arab Emirates""",100
"""Nigeria""",100
"""Ghana""",100
…,…
"""Namibia""",100
"""Cameroon""",100
"""Nauru""",100


In [134]:
countries_ref_df.map_rows(lambda x: process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)[0])

map
str
"""Ivory Coast"""
"""Ivory Coast"""
"""United Arab Emirates"""
"""Nigeria"""
"""Ghana"""
…
"""Namibia"""
"""Cameroon"""
"""Nauru"""


In [135]:
countries_ref_df.map_rows(lambda x: process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio),x[0])

TypeError: argument 'output_type': 'str' is not a Polars data type

In [136]:
x = ['England','The offical state of England','Englaend','Eng land']

In [137]:
process.extract('England',x,scorer=fuzz.partial_ratio)

[('England', 100),
 ('The offical state of England', 100),
 ('Englaend', 86),
 ('Eng land', 86)]

In [138]:
process.extract('England',x,scorer=fuzz.partial_ratio)

[('England', 100),
 ('The offical state of England', 100),
 ('Englaend', 86),
 ('Eng land', 86)]

In [139]:
process.extractOne('England',x,scorer=fuzz.partial_ratio)

('England', 100)

In [140]:
x = ['England','The offical state of England','Englaend']

In [141]:
x = ['England','The offical state of England','Englaend']

In [142]:
x = ['Englasand','The offical state of England','Englaend']

In [143]:
process.extractOne('England',x,scorer=fuzz.partial_ratio)

('The offical state of England', 100)

In [144]:
process.extractOne('England',x,scorer=fuzz.partial_ratio) + 'England'

TypeError: can only concatenate tuple (not "str") to tuple

In [145]:
process.extractOne('England',x,scorer=fuzz.partial_ratio)[0] + 'England'

'The offical state of EnglandEngland'

In [146]:
process.extractOne('England',x,scorer=fuzz.partial_ratio)[0] + ' England'

'The offical state of England England'

In [147]:
countries_ref_df.map_rows(lambda x: process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio),x[0])

TypeError: argument 'output_type': 'str' is not a Polars data type

In [148]:
countries_ref_df.map_rows(lambda x: process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio))

column_0,column_1
str,i64
"""Ivory Coast""",100
"""Ivory Coast""",100
"""United Arab Emirates""",100
"""Nigeria""",100
"""Ghana""",100
…,…
"""Namibia""",100
"""Cameroon""",100
"""Nauru""",100


In [149]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)[0] + 'x[0]')

SyntaxError: incomplete input (<ipython-input-149-e9a48d8fb09c>, line 1)

In [150]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)[0]) + 'x[0]')

map
str
"""Ivory Coastx[0]"""
"""Ivory Coastx[0]"""
"""United Arab Emiratesx[0]"""
"""Nigeriax[0]"""
"""Ghanax[0]"""
…
"""Namibiax[0]"""
"""Cameroonx[0]"""
"""Naurux[0]"""


In [151]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)[0]) + x[0])

map
str
"""Ivory CoastIvory Coast"""
"""Ivory CoastIvory Coast"""
"""United Arab EmiratesUnited Ara…"
"""NigeriaNigeria"""
"""GhanaGhana"""
…
"""NamibiaNamibia"""
"""CameroonCameroon"""
"""NauruNauru"""


In [152]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)[0]) + ';' + x[0])

map
str
"""Ivory Coast;Ivory Coast"""
"""Ivory Coast;Ivory Coast"""
"""United Arab Emirates;United Ar…"
"""Nigeria;Nigeria"""
"""Ghana;Ghana"""
…
"""Namibia;Namibia"""
"""Cameroon;Cameroon"""
"""Nauru;Nauru"""


In [153]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio).filter(pl.col('column_1') < 100)[0]) + ';' + x[0])

AttributeError: 'tuple' object has no attribute 'filter'

In [154]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)[0]) + ';' + x[0])

map
str
"""Ivory Coast;Ivory Coast"""
"""Ivory Coast;Ivory Coast"""
"""United Arab Emirates;United Ar…"
"""Nigeria;Nigeria"""
"""Ghana;Ghana"""
…
"""Namibia;Namibia"""
"""Cameroon;Cameroon"""
"""Nauru;Nauru"""


In [155]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)) + ';' + x[0])

TypeError: can only concatenate tuple (not "str") to tuple

In [156]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)) + ';' + (x[0]))

TypeError: can only concatenate tuple (not "str") to tuple

In [157]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)) +  (x[0]))

TypeError: can only concatenate tuple (not "str") to tuple

In [158]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)) +  tuple(x[0]))

ShapeError: could not create a new DataFrame: series "column_0" has length 262 while series "column_6" has length 250

In [159]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)) + tuple(x[0]))

ShapeError: could not create a new DataFrame: series "column_0" has length 262 while series "column_6" has length 250

In [160]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)) + ';' + x[0]))

SyntaxError: unmatched ')' (<ipython-input-160-e3c8db21c417>, line 1)

In [161]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)) + ';' + (x[0]))

TypeError: can only concatenate tuple (not "str") to tuple

In [162]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)[0]) + ';' + x[0])

map
str
"""Ivory Coast;Ivory Coast"""
"""Ivory Coast;Ivory Coast"""
"""United Arab Emirates;United Ar…"
"""Nigeria;Nigeria"""
"""Ghana;Ghana"""
…
"""Namibia;Namibia"""
"""Cameroon;Cameroon"""
"""Nauru;Nauru"""


In [163]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)) + ';' + x[0]))

SyntaxError: unmatched ')' (<ipython-input-163-e3c8db21c417>, line 1)

In [164]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)[0]) + ';' + x[0])

map
str
"""Ivory Coast;Ivory Coast"""
"""Ivory Coast;Ivory Coast"""
"""United Arab Emirates;United Ar…"
"""Nigeria;Nigeria"""
"""Ghana;Ghana"""
…
"""Namibia;Namibia"""
"""Cameroon;Cameroon"""
"""Nauru;Nauru"""


In [165]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)[0]) + ';' + x[0]).split(';')

AttributeError: 'DataFrame' object has no attribute 'split'

In [166]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)[0]) + ';' + x[0]).select(pl.col('map').str.split(';'))

map
list[str]
"[""Ivory Coast"", ""Ivory Coast""]"
"[""Ivory Coast"", ""Ivory Coast""]"
"[""United Arab Emirates"", ""United Arab Emirates""]"
"[""Nigeria"", ""Nigeria""]"
"[""Ghana"", ""Ghana""]"
…
"[""Namibia"", ""Namibia""]"
"[""Cameroon"", ""Cameroon""]"
"[""Nauru"", ""Nauru""]"


In [167]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)[0]) + ';' + x[0]).select(pl.col('map').str.split(';')).arr_to_struct()

AttributeError: 'DataFrame' object has no attribute 'arr_to_struct'

In [168]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)[0]) + ';' + x[0]).select(pl.col('map').str.split(';').arr_to_struct())

AttributeError: 'Expr' object has no attribute 'arr_to_struct'

In [169]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)[0]) + ';' + x[0]).select(pl.col('map').str.split(';').arr_to_struct(n_field_strategy="max_width"))

AttributeError: 'Expr' object has no attribute 'arr_to_struct'

In [170]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)[0]) + ';' + x[0]).select(pl.col('map').str.split(';').arr.to_struct(n_field_strategy="max_width"))

TypeError: ExprArrayNameSpace.to_struct() got an unexpected keyword argument 'n_field_strategy'

In [171]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)[0]) + ';' + x[0]).select(pl.col('map').str.split(';').arr.to_struct().unnest())

AttributeError: 'Expr' object has no attribute 'unnest'

In [172]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)[0]) + ';' + x[0]).select(pl.col('map').str.split(';').arr.to_struct()).unnest()

PanicException: Only array dtype is expected for `arr.to_struct`.

In [173]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)[0]) + ';' + x[0]).select(pl.col('map').str.split(';').arr.to_struct()).unnest('map')

PanicException: Only array dtype is expected for `arr.to_struct`.

In [174]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)[0]) + ';' + x[0]).select(pl.col('map').str.split(';'))

map
list[str]
"[""Ivory Coast"", ""Ivory Coast""]"
"[""Ivory Coast"", ""Ivory Coast""]"
"[""United Arab Emirates"", ""United Arab Emirates""]"
"[""Nigeria"", ""Nigeria""]"
"[""Ghana"", ""Ghana""]"
…
"[""Namibia"", ""Namibia""]"
"[""Cameroon"", ""Cameroon""]"
"[""Nauru"", ""Nauru""]"


In [175]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)[0]) + ';' + x[0]).select(pl.col('map').str.split(';'))[23]

map
list[str]
"[""Greece"", ""Greece""]"


In [176]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)[0]) + ';' + x[0]).select(pl.col('map').str.split(';'))[69]

map
list[str]
"[""Senegal"", ""Senegal""]"


In [177]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)[0]) + ';' + x[0]).select(pl.col('map').str.split(';'))[156]

map
list[str]
"[""Liberia"", ""Liberia""]"


In [178]:
z = countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)[0]) + ';' + x[0]).select(pl.col('map').str.split(';'))[156]

In [179]:
z = countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)[0]) + ';' + x[0]).select(pl.col('map').str.split(';'))

In [180]:
z = countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)) + ';' + x[0]).select(pl.col('map').str.split(';'))

TypeError: can only concatenate tuple (not "str") to tuple

In [181]:
z = countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio).str) + ';' + x[0]).select(pl.col('map').str.split(';'))

AttributeError: 'tuple' object has no attribute 'str'

In [182]:
z = countries_ref_df.map_rows(lambda x: ''.join(process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)) + ';' + x[0]).select(pl.col('map').str.split(';'))

TypeError: sequence item 1: expected str instance, int found

In [183]:
z = countries_ref_df.map_rows(lambda x: ''.join(process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)) ).select(pl.col('map').str.split(';'))

TypeError: sequence item 1: expected str instance, int found

In [184]:
z = countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio).str) + ';' + x[0]).select(pl.col('map').str.split(';'))

AttributeError: 'tuple' object has no attribute 'str'

In [3]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)[0]) + ';' + x[0]).select(pl.col('map').str.split(';'))[156]

NameError: name 'countries_ref_df' is not defined

In [186]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)[0]) + ';' + x[0]).select(pl.col('map').str.split(';'))

map
list[str]
"[""Ivory Coast"", ""Ivory Coast""]"
"[""Ivory Coast"", ""Ivory Coast""]"
"[""United Arab Emirates"", ""United Arab Emirates""]"
"[""Nigeria"", ""Nigeria""]"
"[""Ghana"", ""Ghana""]"
…
"[""Namibia"", ""Namibia""]"
"[""Cameroon"", ""Cameroon""]"
"[""Nauru"", ""Nauru""]"


In [187]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)[0],cutoff=90) + ';' + x[0]).select(pl.col('map').str.split(';'))

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (<ipython-input-187-c1c38bd555d4>, line 1)

In [188]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)[0],cutoff==90) + ';' + x[0]).select(pl.col('map').str.split(';'))

NameError: name 'cutoff' is not defined

In [2]:
countries_ref_df.map_rows(lambda x: (process.extractOne(x[0],flags_df['country'],scorer=fuzz.token_set_ratio)[0],score_cutoff==90) + ';' + x[0]).select(pl.col('map').str.split(';'))

NameError: name 'countries_ref_df' is not defined